In [4]:
import os
import pandas as pd
import sys
print(sys.path)
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
import sys
from finrl.config import INDICATORS
from stable_baselines3 import SAC
# sys.path.append('/insomnia001/depts/free/users/ik2592/finrl')

from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
import os
import pandas as pd
import matplotlib.pyplot as plt


['/Users/ryansherby/opt/anaconda3/envs/pyML/lib/python311.zip', '/Users/ryansherby/opt/anaconda3/envs/pyML/lib/python3.11', '/Users/ryansherby/opt/anaconda3/envs/pyML/lib/python3.11/lib-dynload', '', '/Users/ryansherby/opt/anaconda3/envs/pyML/lib/python3.11/site-packages']


/Users/ryansherby/opt/anaconda3/envs/pyML/lib/python3.11/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [1]:
from finrl.config import INDICATORS

INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [13]:
df = pd.read_csv('data/BTC-USD.csv').rename(columns = str.lower)
df['tic'] = 'BTC-USD'

In [14]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_turbulence=True,
    user_defined_feature=False
)

In [15]:
proc = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [16]:
proc

,date,adj close,close,high,low,open,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,21056800,BTC-USD,0.000000,487.406167,394.367850,0.000000,-66.666667,100.000000,457.334015,457.334015,0.000000
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,34483200,BTC-USD,-0.738007,487.406167,394.367850,0.000000,-66.666667,100.000000,440.887009,440.887009,0.000000
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,37919700,BTC-USD,-1.859626,488.089504,362.957168,0.000000,-100.000000,100.000000,425.523336,425.523336,0.000000
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,36863600,BTC-USD,-1.814822,475.088945,367.648055,19.194166,-58.483084,100.000000,421.368500,421.368500,0.000000
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,26580100,BTC-USD,-2.153494,467.565274,366.152732,16.808812,-63.110875,100.000000,416.859003,416.859003,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,2024-12-27,94164.859375,94164.859375,97294.843750,93310.742188,95704.976562,52419934565,BTC-USD,252.545647,106303.592116,92306.550852,52.660929,-101.626977,16.823882,98667.262760,91380.854818,0.517394
3755,2024-12-28,95163.929688,95163.929688,95525.898438,94014.289062,94160.187500,24107436185,BTC-USD,41.713616,106172.000914,91830.933461,53.552912,-100.623061,16.823882,98650.978125,91754.912109,0.108877
3756,2024-12-29,93530.226562,93530.226562,95174.875000,92881.789062,95174.054688,29635885267,BTC-USD,-254.267439,106358.834829,91253.850327,51.898691,-121.296875,20.247529,98519.934896,92108.090234,0.518399
3757,2024-12-30,92643.210938,92643.210938,94903.320312,91317.132812,93527.195312,56188003691,BTC-USD,-554.022875,106599.086539,90610.376742,51.013620,-136.135421,24.817878,98393.073438,92481.890625,0.181781


In [ ]:
import yfinance as _yf
import pandas as pd


def yfinance_daily_ohlcv(
    ticker: str,
    start: str | None = None,
    end: str | None = None,
) -> pd.DataFrame:
    """
    Fetch daily OHLCV candles via yFinance.

    `ticker` should follow Yahoo’s format, e.g. 'BTC-USD', 'ETH-USD'.
    """
    if _yf is None:
        raise ImportError("yfinance is not installed. Run: pip install yfinance")

    raw = _yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=False)
    raw.columns = raw.columns.droplevel("Ticker")
    raw = raw.reset_index()
    raw.columns.name = None
    

    return raw

In [2]:
COINS = ["btc"]

DEVICE = "mps"

DATA_DIR    = "./data"
OUTPUT_ROOT = "./models"

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_turbulence=True,
    user_defined_feature=False
)
common_env_kwargs = {
    "hmax": 100,
    "initial_amount": 1_000_000,
    "buy_cost_pct": None,  # filled per-ticker below
    "sell_cost_pct": None, # "
    "num_stock_shares": None, # "
    "reward_scaling": 1e-4
}

In [3]:
os.makedirs(OUTPUT_ROOT, exist_ok=True)

model_ppo = None
agent     = None
policy_kwargs = dict(
    dueling=True,
    net_arch=[512, 256],
)
all_returns = {}
metrics = [] 

In [ ]:
for idx, coin in enumerate(COINS):
    print(f"\n--- Epoch {idx+1}/{len(COINS)}: Training on {coin.upper()} ---")

    # a) load & rename
    df = pd.read_csv(f"{DATA_DIR}/{coin}.csv").rename(columns={
        "Start":     "date",
        "Close": "adjcp",
        "Close":     "close",
        "High":      "high",
        "Low":       "low",
        "Volume":    "volume",
        "Open":      "open",
    })
    df["tic"] = coin

    # b) featurize
    proc = fe.preprocess_data(df)

    # c) split
    train = data_split(proc, "2013-01-01", "2023-12-31")
    # (you can skip making trade set here if you’re not evaluating yet)

    # d) env kwargs
    stock_dim   = len(train.tic.unique())      # will be 1
    state_space = 1 + 2*stock_dim + len(INDICATORS)*stock_dim
    env_kwargs = {
        "hmax":               100,
        "initial_amount":     1_000_000,
        "buy_cost_pct":       [0.001]*stock_dim,
        "sell_cost_pct":      [0.001]*stock_dim,
        "num_stock_shares":   [0]*stock_dim,
        "stock_dim":          stock_dim,
        "action_space":       stock_dim,
        "state_space":        state_space,
        "tech_indicator_list": INDICATORS,
        "reward_scaling":      1e-4,
    }

    

    # e) make SB3 env
    env_train, _ = StockTradingEnv(df=train, **env_kwargs).get_sb_env()

    if idx == 0:
        # first ticker → instantiate agent + model
        agent   = DRLAgent(env=env_train)
        # model_ppo = agent.get_model("ppo")
        # model_ppo  = SAC(
        #     "MlpPolicy",
        #     env_train,
        #     verbose=0,
        #     gamma = 0.95,
        #     device="cuda",
        #     tensorboard_log=OUTPUT_ROOT,
        #     policy_kwargs={
        #         "net_arch": [512, 256],
        #         # optional: choose activation, etc.
        #     },
        # )
        SAC_PARAMS = {
            "batch_size": 128,
            "buffer_size": 100000,
            "learning_rate": 0.0003,
            "learning_starts": 100,
            "ent_coef": "auto_0.1",
            "device": DEVICE,
        }
        model_ppo = agent.get_model("sac",model_kwargs = SAC_PARAMS)

        # set up logging once
        logger = configure(OUTPUT_ROOT, ["stdout", "csv", "tensorboard"])
        model_ppo.set_logger(logger)

    else:
        # reuse the same model, just point it at the new env
        model_ppo.set_env(env_train)

    # f) train (will continue from previous weights)
    # model_ppo = agent.train_model(
    #     model=model_ppo,
    #     tb_log_name=f"{tic}_ppo",
    #     total_timesteps=50000,   # adjust per‐ticker budget
    # )
    # model_ppo.learn(
    # total_timesteps=50000,
    # reset_num_timesteps=False,)
    
    trained_sac = agent.train_model(model=model_ppo, 
                             tb_log_name='sac',
                             total_timesteps=80000)



    trade = data_split(proc, "2024-01-01", "2025-01-01")
    e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)
    df_daily_return, df_actions = DRLAgent.DRL_prediction(
        model=trained_sac,
        environment=e_trade_gym
    )

    # store the test curve
    all_returns[coin] = df_daily_return.copy()
    metrics.append({
        "ticker": coin,
        "end_total_asset": df_daily_return["account_value"].iloc[-1]
    })

    # g) checkpoint after each ticker

    ckpt = os.path.join(OUTPUT_ROOT, f"dqn_after_{coin}.zip")
    model_ppo.save(ckpt)
    print(f"  ↳ checkpoint saved to {ckpt}")

print("\n✅ All tickers trained sequentially on one shared PPO model.")
print(metrics)


--- Epoch 1/1: Training on BTC ---
Successfully added technical indicators
Successfully added turbulence index
{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1', 'device': 'mps'}
Using mps device
Logging to ./models


KeyboardInterrupt: 

In [ ]:
# Path to save
out_path = os.path.join(OUTPUT_ROOT, "all_tickers_equity_curves.png")

# Grab the “date” series from one of the tickers and parse as datetime
example_dates = pd.to_datetime(next(iter(all_returns.values()))["date"])
first_date = example_dates.iloc[0]
last_date  = example_dates.iloc[-1]

# Plot
fig, ax = plt.subplots(figsize=(12, 8))
for ticker, df_ret in all_returns.items():
    # ensure we plot datetime on x-axis
    dates = pd.to_datetime(df_ret["date"])
    ax.plot(dates, df_ret["account_value"], label=ticker)

# Only set ticks at first & last date
ax.set_xticks([first_date, last_date])
ax.set_xticklabels(
    [first_date.strftime("%Y-%m-%d"), last_date.strftime("%Y-%m-%d")],
    rotation=45,
    ha="right"
)

ax.set_xlabel("Date")
ax.set_ylabel("Account Value")
ax.set_title("Test Equity Curves by Ticker")
ax.legend(ncol=2, fontsize="small")
plt.tight_layout()
plt.savefig(out_path)
print(f"↳ Saved combined equity-curve plot to {out_path}")

# 2) (Optional) bar‐plot of final asset
df_metrics = pd.DataFrame(metrics)
plt.figure(figsize=(10, 6))
plt.bar(df_metrics["ticker"], df_metrics["end_total_asset"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("End Total Asset")
plt.title("Final Portfolio Value by Ticker")
plt.tight_layout()
out_path2 = os.path.join(OUTPUT_ROOT, "final_asset_by_ticker.png")
plt.savefig(out_path2)
print(f"↳ Saved bar‐plot of end‐period assets to {out_path2}")

↳ Saved combined equity-curve plot to ./models/all_tickers_equity_curves.png
↳ Saved bar‐plot of end‐period assets to ./models/final_asset_by_ticker.png
